# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [195]:
%store -r train_data_set
%store -r train_labels
%store -r test_data_set
%store -r test_labels
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target

# data_set = data_set[labels!=2]
# labels = labels[labels!=2]

labels[labels<1] = -1

In [196]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        d = 2 #quadratic case, for different cases change the value
        kernel = np.dot(data_set[:-1].T * data_set[-1]) ** d
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [197]:
def get_labels_count(data_set):
    return len(np.unique(labels))

Use the code that we have implemented earlier:

In [198]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-05):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    print(A)
    print(f'A={np.linalg.matrix_rank(A)}, P={np.linalg.matrix_rank(P)} G={np.linalg.matrix_rank(G)}\n h={np.linalg.matrix_rank(h)} b={b}')

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])
    print(lambdas)
    support_vectors_id = np.where(lambdas > threshold)[0]
    print(f' svi = {support_vectors_id}')
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)
    print(f'len(lambdas) {len(lambdas)}, b={b}')

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [199]:
def choose_set_for_label(data_set, label):
    one_on_one_sets = []
    one_on_one_labels = []
    for l in range(get_labels_count(data_set)):
        print(l)
        if l == 0:
            l = -1
        if l == label:
            label_set_labels = labels[labels==l]
            label_dataset = data_set[labels==l]
            # print(new_data_labels)
        else:
            one_on_one_sets.append(data_set[labels==l])
            one_on_one_labels.append(labels[labels==l])
    # print(f'{np.shape(one_on_one_sets)}\n\n\n{np.shape(new_data_labels)}')
    train_parameters = []
    for i in range(len(one_on_one_sets)):
        ds = np.concatenate((label_dataset, one_on_one_sets[i]))
        ls = np.concatenate((label_set_labels, one_on_one_labels[i]))
        ls[ls==ls.min()] = -1
        ls[ls==ls.max()] = 1
        train_data_set, test_data_set, train_labels, test_labels = train_test_split(
            ds, ls, test_size=0.2, random_state=15)
        train_parameters.append((train_data_set, test_data_set, train_labels, test_labels))
    return train_parameters

In [210]:
# modify this part

scores = []
for i in range(get_labels_count(data_set)):
    if i == 0:
        i = -1
    train_parameters = choose_set_for_label(data_set, i)
    for parameter in train_parameters:
        train_data_set, test_data_set, train_labels, test_labels = parameter
        objects_count = len(train_labels)
        lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf', threshold=1e-04)
        predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
        predicted = list(predicted.astype(int))
        scores.append(accuracy_score(predicted, test_labels))
        print(f'[0--1, 0--2, 1--0, 1--2, 2--0, 2--1]\n{scores}')

0
1
2
[[ 1. -1. -1.  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1.
   1. -1.  1. -1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1. -1. -1. -1. -1.
  -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1. -1. -1.  1. -1. -1.  1.
   1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.
   1.  1. -1. -1. -1. -1. -1.  1.]]
A=1, P=80 G=80
 h=1 b=0.0
     pcost       dcost       gap    pres   dres
 0:  9.6305e+01 -1.2289e+03  2e+03  2e-01  2e-15
 1:  5.9143e+01 -1.2031e+02  2e+02  5e-03  2e-15
 2:  7.0898e+00 -1.6497e+01  2e+01  2e-16  2e-15
 3: -5.2057e-01 -3.7668e+00  3e+00  2e-16  9e-16
 4: -1.1712e+00 -1.8374e+00  7e-01  2e-16  3e-16
 5: -1.3952e+00 -1.6846e+00  3e-01  2e-16  2e-16
 6: -1.4671e+00 -1.5679e+00  1e-01  2e-16  2e-16
 7: -1.5060e+00 -1.5164e+00  1e-02  2e-16  2e-16
 8: -1.5105e+00 -1.5106e+00  1e-04  2e-16  2e-16
 9: -1.5105e+00 -1.5105e+00  1e-06  2e-16  2e-16
Optimal solution found.
[[7.14889917e-08]
 [5.47355009e-08]
 [5.58727686e-08]
 [8.91950887e-08]
 [